In [1]:
from nsga import NSGA
from moga import MOGA
import networkx as nx
import itertools
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
from scipy.spatial import ConvexHull

In [2]:
# Carga del grafo
graph = nx.read_graphml("data/amazon_graph.graphml")

# Renombramos los nodos al rango [0, N]
mapping = {node: i for i, node in enumerate(graph.nodes())}
graph = nx.relabel_nodes(graph, mapping)


In [3]:
def run_experiment(combination, num_experiments=5, algorithm="NSGA"):
    area_sum = 0
    max_area = 0
    best_front_individuals = None

    for _ in range(num_experiments):
        if algorithm == "NSGA":
            estrategia = NSGA(graph, *combination)
        elif algorithm == "MOGA":
            estrategia = MOGA(graph, *combination)
        old_pop, fitness, pareto = estrategia.evolve()

        # Seleccionamos los valores de fitness del frente de pareto
        selected_fitness = [fitness[i] for i in pareto]

        # Calculamos el area bajo los puntos del frente de pareto
        points = np.array(selected_fitness) 

        # Añadir (0,0) como punto de referencia
        points = np.vstack((points, [0, 0]))

        # Minimo 3 puntos para calcular convex hull
        if len(points) > 2:
            hull = ConvexHull(points)
            # Calcular area bajo el convex Hull
            area = hull.area

        # Si hay menos de 3 puntos (incluyendo (0, 0)), el area es 0
        else:
            area = 0
    
        if area > max_area:
            # Guardamos el area mas grande
            max_area = area
            # Guardamos la poblacion con el area mas grande
            best_front_individuals = [old_pop[i] for i in pareto]
            
        area_sum += area

    average_area = area_sum / num_experiments
    
    return combination, average_area, max_area, best_front_individuals

In [5]:
#NSGA-II
N = [80]
init = [0.4]
pcross= [0.6]
pmut = [0.4]
n_iter= [625]
fitness_metrics= [0,1,2,3]
n_tour = [2]
crossover_op=[2]

In [13]:
# MOGA
N = [100]
init = [0.5]
pcross= [0.55]
pmut = [0.4]
n_iter= [500]
fitness_metrics= [0,1,2,3]
n_tour = [2]
crossover_op=[2]

sigma = [0.7]

### Concurrent

In [6]:
# Define el número de veces que cada configuración será ejecutada
num_experiments = 10

# Define el algoritmo a utilizar
algorithm = "NSGA"

# Crear todas las combinaciones posibles de hiperparámetros
all_combinations = list(itertools.product(
    N, init, pcross, pmut, n_iter, fitness_metrics, n_tour, crossover_op
))

# Almacenar los resultados
results = []

# Paralelizar la ejecución
with ProcessPoolExecutor() as executor:
    # Iniciar las ejecuciones en paralelo
    future_to_combination = {executor.submit(run_experiment, combination, num_experiments, algorithm): combination for combination in all_combinations}

    total_combinations = len(all_combinations)
    completed_combinations = 0
    
    for future in as_completed(future_to_combination):
        combination, average_area, max_area, best_front_individuals = future.result()
        results.append((combination, average_area, max_area, best_front_individuals))

        completed_combinations += 1
        print(f"Combinación completada. Progreso: {completed_combinations}/{total_combinations}")
        print(f"Combinación de parámetros: {combination}")
        print(f"Área media de paretos: {average_area}")
        print(f"Pareto con área máxima: {max_area}")

Combinación completada. Progreso: 1/4
Combinación de parámetros: (80, 0.4, 0.6, 0.4, 625, 2, 2, 2)
Área media de paretos: 432.70174650007664
Pareto con área máxima: 437.9551996011397
Combinación completada. Progreso: 2/4
Combinación de parámetros: (80, 0.4, 0.6, 0.4, 625, 3, 2, 2)
Área media de paretos: 433.1020530297531
Pareto con área máxima: 442.18889587573517
Combinación completada. Progreso: 3/4
Combinación de parámetros: (80, 0.4, 0.6, 0.4, 625, 0, 2, 2)
Área media de paretos: 436.7840847366086
Pareto con área máxima: 442.82280031807085
Combinación completada. Progreso: 4/4
Combinación de parámetros: (80, 0.4, 0.6, 0.4, 625, 1, 2, 2)
Área media de paretos: 66.37600288811288
Pareto con área máxima: 67.41672943638204


### Non concurrent

In [ ]:
# Define el número de veces que cada configuración será ejecutada
num_experiments = 2
results = []

# Crear todas las combinaciones posibles de hiperparámetros
all_combinations = list(itertools.product(
    N, init, pcross, pmut, n_iter, fitness_metrics, n_tour, crossover_op, sigma
))

total_combinations = len(all_combinations)
completed_combinations = 0

best_average_area = 0
best_average_combination = None

best_max_area = 0
best_max_combination = None


for combination in all_combinations:
    combination, average_area, max_area, best_front_individuals = run_experiment(combination, num_experiments)

    results.append((combination, average_area, max_area, best_front_individuals))
    completed_combinations += 1
    print(f"Combinación completada. Progreso: {completed_combinations}/{total_combinations}")
    print(f"Combinación de parámetros: {combination}")
    print(f"Área media de paretos: {average_area}")
    print(f"Pareto con área máxima: {max_area}")

    if average_area > best_average_area:
        best_average_combination = [combination, average_area, max_area, best_front_individuals]
    
    if max_area > best_max_area:
        best_max_combination = [combination, average_area, max_area, best_front_individuals]